In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import re

CHEMIN = "C:/Users/mathy/OneDrive/Bureau/EthicsFinance" # A changer selon l'endroit où vous clonez le repo git
esg_grade = pd.read_csv(f'{CHEMIN}/Score ESG brut.csv')

In [2]:
esg_grade = esg_grade.dropna(subset=['ISSUER_EQUITY_TICKER', 'Groupe Industrie GICS', 'Score ESG brut'])
esg_grade = esg_grade.drop(columns=['Pays', 'Date de Scoring', 'Identifiant Bloomberg Company', 'Identifiant sous industrie GICS', "Nom de l'émetteur"])
esg_grade["ISSUER_EQUITY_TICKER"] = esg_grade["ISSUER_EQUITY_TICKER"].str.strip().str.replace(r'US$', '', regex=True)

In [ ]:
esg_grade = esg_grade[~esg_grade["ISSUER_EQUITY_TICKER"].str.contains(r'\d')]

In [ ]:
top_60_percent = esg_grade.sort_values("Score ESG brut", ascending=False)
top_60_percent = top_60_percent.head(int(0.6 * len(esg_grade)))


In [ ]:
esg_grade["ISSUER_EQUITY_TICKER"] = esg_grade["ISSUER_EQUITY_TICKER"].str.split().str[0]

In [ ]:

end_date = datetime.today().date()
start_date = end_date - timedelta(days=3*365)

historical_data = {}

for ticker in esg_grade["ISSUER_EQUITY_TICKER"].unique():
    try:
        # Essayer de récupérer les données
        data = yf.download(ticker, start=start_date, end=end_date, interval="1d")
        if data.empty:
            print(f"Aucune donnée pour le ticker {ticker}.")
            continue
        data["Ticker"] = ticker
        historical_data[ticker] = data
    except Exception as e:
        print(f"Erreur pour le ticker {ticker} : {e}")

df_all_prices = pd.concat(historical_data.values())


df_all_prices = df_all_prices.reset_index()

df_all_prices.to_csv(f'{CHEMIN}/price_ticker.csv')
print(df_all_prices.head())


In [8]:
import pandas as pd
import yfinance as yf
fichier_final = pd.read_csv('C:/Users/mathy/OneDrive/Bureau/EthicsFinance/price_ticker.csv')

In [2]:
fichier_final = fichier_final.dropna(axis=1)


In [10]:
fichier_final = fichier_final[-1:].apply(pd.to_numeric, errors='coerce')
daily_returns = fichier_final.pct_change()

# Calcul du rendement moyen pour chaque action
mean_returns = daily_returns.mean()

C:\Users\mathy\AppData\Local\Temp\ipykernel_10648\3638717748.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = fichier_final.pct_change()


In [11]:
sorted_returns = mean_returns.sort_values(ascending=False)

top_n = int(len(sorted_returns) * 0.5)

# Garder les 50% du haut
top_50_percent = sorted_returns.head(top_n)

In [12]:
top_ticker_names = top_50_percent.index.tolist()

In [13]:
top_ticker_names = pd.DataFrame(top_ticker_names)

In [37]:
top_ticker_names.to_csv(f'{CHEMIN}/tickers_finaux.csv')

In [14]:
import pandas as pd
import os

# Spécifie le chemin vers le dossier contenant les fichiers Excel
chemin_dossier = 'C:/Users/mathy/OneDrive/Bureau/Weightings'

# Récupère tous les fichiers Excel dans le dossier
fichiers = [f for f in os.listdir(chemin_dossier) if f.endswith(('.xls', '.xlsx'))]

# Liste pour stocker les DataFrames
liste_df = []

for fichier in fichiers:
    # Extrait le trimestre et l'année à partir du nom du fichier (ex: "1_2020")
    nom_sans_extension = os.path.splitext(fichier)[0]
    date_trimestre = nom_sans_extension  # ou fais un traitement plus fin si besoin

    # Charge le fichier Excel
    chemin_complet = os.path.join(chemin_dossier, fichier)
    df = pd.read_excel(chemin_complet)

    # Ajoute la colonne "date_trimestre"
    df["date_trimestre"] = date_trimestre

    # Ajoute au tableau principal
    liste_df.append(df)

# Concatène tous les DataFrames
df_final = pd.concat(liste_df, ignore_index=True)

# (optionnel) Sauvegarde le résultat
df_final.to_excel('bench_concat.xlsx', index=False)


In [18]:
# Assure que date_trimestre est triée dans l’ordre
df_final['date_trimestre'] = pd.Categorical(df_final['date_trimestre'],
                                             categories=sorted(df_final['date_trimestre'].unique(), key=lambda x: (int(x.split('_')[1]), int(x.split('_')[0]))),
                                             ordered=True)

# Trie par trimestre
df_final = df_final.sort_values('date_trimestre')

# Crée un dictionnaire pour stocker les returns trimestriels
returns = {}

# Liste des trimestres triés
trimestres = df_final['date_trimestre'].cat.categories.tolist()

# Boucle sur les paires de trimestres (t-1, t)
for i in range(1, len(trimestres)):
    t_prev = trimestres[i - 1]
    t_curr = trimestres[i]
    
    df_prev = df_final[df_final['date_trimestre'] == t_prev]
    df_curr = df_final[df_final['date_trimestre'] == t_curr]

    # Merge sur un identifiant commun, ici on suppose qu’il y a une colonne 'asset' (le nom de l'actif)
    df_merged = pd.merge(df_curr, df_prev, on='Ticker', suffixes=('_curr', '_prev'))

    # Calcul du return pondéré du portefeuille
    weighted_return = ((df_merged['Price_curr'] / df_merged['Price_prev']) - 1) * df_merged['Weight_curr']
    
    portfolio_return = weighted_return.sum()
    returns[t_curr] = portfolio_return

# Transforme en DataFrame pour affichage
df_returns = pd.DataFrame(list(returns.items()), columns=['date_trimestre', 'portfolio_return'])
print(df_returns)


   date_trimestre  portfolio_return
0          1_2020        -22.093824
1          2_2020         17.721106
2          3_2020          0.894649
3          4_2020         14.701741
4          1_2021          9.946863
5          2_2021          5.498201
6          3_2021          0.013173
7          4_2021          6.079225
8          2_2022        -16.925947
9          3_2022         -3.805916
10         4_2022         13.259394
11         1_2023         13.238669
12         2_2023          1.671141
13         3_2023         -3.838787
14         4_2023          8.521958
15         1_2024         11.748588
16         2_2024         -2.431711
17         3_2024          5.430800
18         4_2024         -2.218341
19         1_2025          9.488661


In [19]:
df_returns.to_excel('bench_non_esg.xlsx', index=False)

In [31]:
# Lecture de la liste de tickers à conserver
tickers = pd.read_csv("tickers_finaux.csv")  # ou pd.read_csv("tickers.csv")
tickers_faysal = pd.read_excel("bench_concat.xlsx") 

# On suppose que la colonne contenant les tickers s'appelle 'Ticker' dans les deux fichiers





In [32]:
tickers_faysal['Tickers'] = tickers_faysal['Tickers'].str.replace(' Equity', '', regex=False)

In [33]:
df_merged = tickers_faysal.merge(tickers, on='Tickers', how='inner')